# Decision tree will be put on hiatus because I want to do more fun things than fix this :(.

In [2]:
import pandas as pd
from typing import List, Any, Tuple

In [3]:
csvfile = "/Users/alifabdullah/Collaboration/Kaggle-ML-Algorithm-Musings/datasets/car_evaluation.csv"
csv_in_panda_form = pd.read_csv(csvfile)
target_column = "Decision"
feature_columns = [feature_header for feature_header in csv_in_panda_form.columns.drop(target_column)]

#print(csv_in_panda_form)
#print(feature_columns)

#for row in csv_in_panda_form.iterrows():
#  print(row[1]["Outcome"])

print(len(csv_in_panda_form[csv_in_panda_form["NumberOfDoors"] == '2']))
print((csv_in_panda_form["Decision"].unique()))
print(csv_in_panda_form.iloc[0])
  


432
['unacc' 'acc' 'vgood' 'good']
BuyingPrice        vhigh
MaintenanceCost    vhigh
NumberOfDoors          2
NumberOfPersons        2
LugBoot            small
Safety               low
Decision           unacc
Name: 0, dtype: object


In [4]:
class Branch:
  """
  This class represents a branch from a Decision tree.
  """
  
  def __init__(self):
    self.value = None
    self.next_node : 'DecisionTreeNode' = None

  def change_value(self, value):
    self.value = value

  def change_next_node(self, next_node: 'DecisionTreeNode'):
    self.next_node = next_node

class DecisionTreeNode:
  """
  This class represents the nodes that will make up my decision tree.
  There are two types of nodes. Branching nodes - whose children are either
  more branching nodes or leaf nodes - and leaf nodes - where, upon reaching these
  nodes, you end up with a label for the test instance.
  """

  def __init__(self):
    self.list_of_branches : List[Branch] = [] # Branch Node only
    self.label = None # Leaf Node only
    self.feature_name = None # Branch Node only
  
  # Creates a Branch Node
  def add_branch(self, branch: Branch):
    self.list_of_branches.append(branch)
  
  # Creates a Leaf Node
  def change_label(self, label):
    self.label = label
  
  # Changes the feature name of a Branch Node
  def assign_feature_name(self, feature_name):
    # Ensure immutability for feature name once we assign it
    assert self.feature_name is None, "Cannot reassign feature name."

    self.feature_name = feature_name

In [61]:
def _decision_tree_maker(dataframe : pd.DataFrame, list_of_labels, features_to_consider, target_column):
  head_node = DecisionTreeNode()
  #print("what???",list_of_labels)
  #print("super huh?",list_of_labels[0])
  # Base Case 1: If our dataset is empty or the error is 0, meaning all labels are the same
  if len(dataframe) == 0:
    #print("skibidy",list_of_labels[0])
    head_node.change_label(list_of_labels[0]) # There is no data to make a label with, so just pick one.
    return head_node
  
  # Base Case 2: The minority label has instance count 0, meaning all labels are the same in this dataset
  if _dataset_error_calculator(dataframe, list_of_labels, target_column) == 0:
    #print("super bwuh",dataframe[target_column])
    #print("super huh?",list_of_labels[0])
    #print("Dpnt get it1",head_node.label)
    head_node.change_label(dataframe[target_column].values[0])

    #print("dont get it",head_node.label)
    return head_node
  
  # Induction step
  else:
    biggest_error_diff = 0
    feature_with_biggest_error_diff = None

    # Compare error before any feature split with each feature split's error:
    error_before_split = _dataset_error_calculator(dataframe, list_of_labels, target_column)
    #print(f"Before feature split: {error_before_split}")

    # Loop over every feature to get its error on the dataframe
    for feature in features_to_consider:
      feature_error = _dataset_error_calculator(dataframe, list_of_labels, target_column, feature)
      #print(f"{feature}: {feature_error}")

      # Find the feature split that causes the biggest error difference from before to after the split
      if error_before_split - feature_error > biggest_error_diff:
        biggest_error_diff = error_before_split - feature_error
        feature_with_biggest_error_diff = feature
    
    # Assign the current node to have the feature that causes the biggest error 
    # difference from before to after the split
    head_node.assign_feature_name(feature_with_biggest_error_diff)
    features_left = [feature_left for feature_left in features_to_consider if feature_left != feature_with_biggest_error_diff]
    #print(f"All features before split: {features_to_consider}")
    list_of_unique_value_datasets, list_of_unique_values = _dataset_splitter_using_feature(dataframe, feature_with_biggest_error_diff)
    #print(f"Features left: {features_left}")
    for unique_value_dataset, unique_value in zip(list_of_unique_value_datasets, list_of_unique_values):

      # Fixing how we add branches to our decision tree implementation.
      current_branch = Branch()
      current_branch.change_value(unique_value)
      #print(unique_value)
      next_decision_tree_node = _decision_tree_maker(unique_value_dataset, list_of_labels, features_left, target_column)
      #print(next_decision_tree_node.label)
      current_branch.change_next_node(next_decision_tree_node)

      head_node.add_branch(current_branch)
    return head_node
      
    




def _dataset_splitter_using_feature(dataframe, feature) -> Tuple[List[Any], List[Any]]:
  """
  Takes in a feature, and then returns a list of dataset splits, one for each unique
  value of the feature, where each split has the same value for the column 'feature'
  """
  list_of_dataset_splits = []
  unique_values = dataframe[feature].unique()
  for unique_value in unique_values:
    list_of_dataset_splits.append(dataframe[dataframe[feature]==unique_value])
  return list_of_dataset_splits, unique_values

def _dataset_error_calculator(dataframe, list_of_labels, target_column, feature=None):
  """
  If you pass in a feature, get the error difference between the dataset before 
  the feature split, and after the feature split.
  """
  if feature is not None:
    unique_feature_values = dataframe[feature].unique()
    error_sum = 0
    
    # Go through every unique value of the chosen feature
    for feature_value in unique_feature_values:

      # Get all rows where the current feature value is present for the feature column.
      # Aggregate the sum of the errors in error_sum
      current_df = dataframe[dataframe[feature] == feature_value]
      current_df_error, label_to_count_mapping_dictionary_current_df = _dataset_consistency_metrics(current_df, list_of_labels, target_column)
      error_sum += current_df_error

    return error_sum
  else:

    # After looking over every label, return the label that has the least amount of
    # instances associated with it.
    error, label_to_count_mapping_dictionary = _dataset_consistency_metrics(dataframe, list_of_labels, target_column)
    return error
    

def _dataset_consistency_metrics(dataframe, list_of_labels, target_column):
  """
  Tells you the counts of instances, in a dataset, corresponding to labels, as 
  well as the error of the dataset, where the error is calculated as the number
  of dataset instances corresponding to the label with the least instances.

  Args:
    dataframe - A Pandas DataFrame, representing our dataset
    list_of_labels - List of labels that represents all labels that could be assigned (replaced by self.list_of_labels going forward)
    target_column: The name of the column we are trying to predict (replaced by self.target_column going forward)
  
  Returns:
    error - The error, as defined above
    label_counts - A dictionary mapping between labels and their respective counts
  """
  label_count_mapping_dict = dict()
  for key in list_of_labels:
    label_count_mapping_dict[key] = 0
  
  for row in dataframe.iterrows():
    label_count_mapping_dict[row[1][target_column]] += 1 
  
  error = float("inf")
  for key in list_of_labels:
    if label_count_mapping_dict[key] < error:
      error = label_count_mapping_dict[key]
  return error, label_count_mapping_dict

# print(_dataset_error_calculator(csv_in_panda_form, ['unacc','acc','vgood','good'], "Decision", None))
# print(_dataset_error_calculator(csv_in_panda_form, ['unacc','acc','vgood','good'],"Decision", "NumberOfDoors"))

# print(_decision_tree_split_decider(csv_in_panda_form, ['unacc','acc','vgood','good'], "Decision"))

all_features = csv_in_panda_form.drop(columns=["Decision"]).columns.to_list()
constructed_decision_tree = _decision_tree_maker(csv_in_panda_form, ['unacc','acc','vgood','good'], all_features, "Decision")


def print_the_tree(tree : DecisionTreeNode, level=0):
  current_tree_value = None
  if tree.feature_name is not None:
    current_tree_value = tree.feature_name
  elif tree.label is not None:
    current_tree_value = tree.label
  ret = str(current_tree_value)+"\n"
  #print(ret+ret)
  for branch in tree.list_of_branches:
    ret += ("\t"*(1+level)+str(branch.value) + "---" + print_the_tree(branch.next_node,level+1))
  return ret

print(print_the_tree(constructed_decision_tree))

Safety
	low---unacc
	med---unacc
	high---LugBoot
		small---unacc
		med---NumberOfDoors
			2---unacc
			3---NumberOfPersons
				2---unacc
				4---unacc
				more---unacc
			4---unacc
			5more---unacc
		big---unacc



Safety
low---	0       unacc
3       unacc
6       unacc
9       unacc
12      unacc
        ...  
1713    unacc
1716    unacc
1719    unacc
1722    unacc
1725    unacc
Name: Decision, Length: 576, dtype: object
med---	1       unacc
4       unacc
7       unacc
10      unacc
13      unacc
        ...  
1714     good
1717     good
1720      acc
1723     good
1726     good
Name: Decision, Length: 576, dtype: object
high---	LugBoot
small---		2       unacc
11      unacc
20      unacc
29      unacc
38      unacc
        ...  
1685     good
1694     good
1703    unacc
1712     good
1721     good
Name: Decision, Length: 192, dtype: object
med---		NumberOfDoors
2---			5       unacc
14      unacc
23      unacc
113     unacc
122     unacc
131     unacc
221     unacc
230       acc
239       acc
329     unacc
338       acc
347       acc
437     unacc
446     unacc
455     unacc
545     unacc
554       acc
563       acc
653     unacc
662       acc
671       acc
761     unacc
770       acc
779       a